Charity Optimization

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("./charity_data.csv")
app_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(["EIN", "NAME"], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df["INCOME_AMT"].value_counts()

In [ ]:
# Determine the number of unique values in each column.
ucounts = df.nunique()
print(ucounts)

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
atcounts = df["APPLICATION_TYPE"].value_counts()
print(atcounts)

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
atcounts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than 500?
replace_application = atcounts.loc[atcounts < 500]
# Replace in dataframe
for app in replace_application:
    df.APPLICATION_TYPE = df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
df.APPLICATION_TYPE.value_counts()
replace_application.head()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_ = df.CLASSIFICATION.value_counts()
df.CLASSIFICATION.value_counts()

In [ ]:
# Visualize the value counts of CLASSIFICATION
class_.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_[class_ < 1000].index)

# Replace in dataframe
for cls in replace_class:
    df.CLASSIFICATION = df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = df.dtypes[df.dtypes == "object"].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(application_cat,1)
df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df.IS_SUCCESSFUL.values
X = df.drop(columns=["IS_SUCCESSFUL"]).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")